<a href="https://colab.research.google.com/github/renan-peres/mfin-portfolio-management/blob/main/02_bond_portfolio_contruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bond Portfolio Selection
Bond selection with convexity > 1 and higher price sensitivity to changes in interest rates.

### Step 1: Import Libraries

In [1]:
# UDFs
from py.utils import load_and_filter_data, export_to_excel
from py.bond_selection import calculate_bond_price, add_bond_prices_to_df, calculate_duration_for_bonds, calculate_modified_duration, calculate_price_change_sensitivity

# Data manipulation libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Excel libraries
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import os

### Step 2: Define Parameters

#### Dates

In [2]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')  # Report date
start_date = end_date - timedelta(days=5*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2020-05-17 to 2025-05-16
Time span: 1825 days (5.00 years)


#### Risk-free rate (T-bill, %)

In [3]:
# Load and process data
daily_risk_free_df = load_and_filter_data('data/datasets/daily_treasury_rates.csv', ['^IRX'], start_date, end_date)
# risk_free_rate = .0433 
risk_free_rate = daily_risk_free_df.iloc[-1, 0] / 100

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")
daily_risk_free_df.head()

Found 1 of 1 tickers in data/datasets/daily_treasury_rates.csv
Missing tickers: []
Risk-Free Rate: 0.0424 -- 13 WEEK TREASURY BILL (^IRX)


,^IRX
Date,
2020-05-18,0.105
2020-05-19,0.118
2020-05-20,0.118
2020-05-21,0.110
2020-05-22,0.110


#### Report File

In [4]:
output_file = f'reports/portfolio-{datetime.date(end_date)}.xlsx'

### Step 3: Import Bond Data

In [5]:
blackrock_etf_df = pd.read_csv('data/datasets/fundamentals_blackrock_bonds.csv')
blackrock_etf_df.head()

,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,AGG,iShares Core U.S. Aggregate Bond ETF,3.16,8.00,1.96,-0.68,1.50,3.10,"Apr 30, 2025","Sep 22, 2003",122782945919,239458,https://www.blackrock.com/us/individual/produc...,0.0478,8.12,0.53,0.0354
1,AGIH,iShares Inflation Hedged U.S. Aggregate Bond ETF,3.42,7.17,-,-,-,2.72,"Apr 30, 2025","Jun 22, 2022",2461365,328179,https://www.blackrock.com/us/individual/produc...,0.0473,7.81,0.51,NaN
2,AGRH,iShares Interest Rate Hedged U.S. Aggregate Bo...,0.77,4.50,-,-,-,5.19,"Apr 30, 2025","Jun 22, 2022",7754408,328180,https://www.blackrock.com/us/individual/produc...,0.0528,8.12,-0.16,NaN
3,AGZ,iShares Agency Bond ETF,2.85,6.98,2.84,0.48,1.74,2.34,"Apr 30, 2025","Nov 05, 2008",608427190,239457,https://www.blackrock.com/us/individual/produc...,0.0430,4.11,0.24,0.0347
4,BAIPX,iShares Short-Term TIPS Bond Index Fund,3.76,7.54,3.19,3.67,-,2.83,"Apr 30, 2025","Feb 16, 2016",5262810,282302,https://www.blackrock.com/us/individual/produc...,NaN,NaN,NaN,NaN


### Step 4: Filter for Convexity > 1

In [6]:
bond_funds_filtered_df = blackrock_etf_df[blackrock_etf_df['Convexity'] >= 1].sort_values(by='Convexity', ascending=False).reset_index(drop=True)

# Define all possible percentage columns
all_percentage_cols = ['YTD (%)', '1Y (%)', '3Y (%)', '5Y (%)', '10Y (%)', 'Incept (%)', 'Since Inception (%)']

# Filter to only include columns that actually exist in the DataFrame
percentage_cols = [col for col in all_percentage_cols if col in bond_funds_filtered_df.columns]

print(f"Processing the following percentage columns: {percentage_cols}")

# First, ensure all percentage columns are converted to numeric values
for col in percentage_cols:
    # Convert to numeric first, handling errors by setting them to NaN
    bond_funds_filtered_df[col] = pd.to_numeric(bond_funds_filtered_df[col], errors='coerce')

# Now perform the division safely
for col in percentage_cols:
    bond_funds_filtered_df[col] = bond_funds_filtered_df[col] / 100

# Format as percentage strings
for col in percentage_cols + ['Yield_To_Maturity', 'Weighted_Avg_Coupon']:
    # Check if column exists and only process if it does
    if col in bond_funds_filtered_df.columns:
        # Only format cells that aren't NaN
        bond_funds_filtered_df[col] = bond_funds_filtered_df[col].apply(
            lambda x: '{:.2%}'.format(x) if pd.notna(x) else x
        )

bond_tickers = bond_funds_filtered_df['Ticker'].tolist()

# Display Outputs
print(f"Number of iShares bond/fixed income funds with Convexity >= 1: {len(bond_tickers)}")
print(bond_tickers)
display(bond_funds_filtered_df.head())

Processing the following percentage columns: ['YTD (%)', '1Y (%)', '3Y (%)', '5Y (%)', '10Y (%)', 'Incept (%)']
Number of iShares bond/fixed income funds with Convexity >= 1: 13
['GOVZ', 'IBGL', 'TLT', 'IBGB', 'ILTB', 'IGLB', 'IBGA', 'TLH', 'ICVT', 'IGOV', 'LQD', 'ELQD', 'LQDI']


,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,GOVZ,iShares 25+ Year Treasury STRIPS Bond ETF,0.65%,1.05%,-12.71%,NaN,NaN,-15.79%,"Apr 30, 2025","Sep 22, 2020",255201565,315911,https://www.blackrock.com/us/individual/produc...,5.01%,27.30,7.24,0.00%
1,IBGL,iShares® iBonds® Dec 2055 Term Treasury ETF,NaN,NaN,NaN,NaN,NaN,NaN,"Apr 30, 2025","Mar 25, 2025",3613848,342146,https://www.blackrock.com/us/individual/produc...,4.91%,29.74,3.48,4.63%
2,TLT,iShares 20+ Year Treasury Bond ETF,3.23%,5.46%,-6.03%,-9.43%,-0.87%,3.79%,"Apr 30, 2025","Jul 22, 2002",48796891937,239454,https://www.blackrock.com/us/individual/produc...,4.97%,25.48,3.38,2.86%
3,IBGB,iShares® iBonds® Dec 2045 Term Treasury ETF,NaN,NaN,NaN,NaN,NaN,NaN,"Apr 30, 2025","Mar 25, 2025",3634667,342124,https://www.blackrock.com/us/individual/produc...,4.97%,19.79,2.34,3.18%
4,ILTB,iShares Core 10+ Year USD Bond ETF,2.20%,6.73%,-1.03%,-3.92%,1.47%,3.87%,"Apr 30, 2025","Dec 08, 2009",591313158,239424,https://www.blackrock.com/us/individual/produc...,5.66%,21.76,2.34,3.94%


In [7]:
# Ensure relevant columns are numeric
bond_funds_filtered_df['Yield_To_Maturity'] = pd.to_numeric(
    bond_funds_filtered_df['Yield_To_Maturity'].str.replace('%', ''), errors='coerce'
)
bond_funds_filtered_df['Weighted_Avg_Coupon'] = pd.to_numeric(
    bond_funds_filtered_df['Weighted_Avg_Coupon'].str.replace('%', ''), errors='coerce'
)
bond_funds_filtered_df['Weighted_Avg_Maturity'] = pd.to_numeric(
    bond_funds_filtered_df['Weighted_Avg_Maturity'], errors='coerce'
)

# Fill missing values with 0
bond_funds_filtered_df = bond_funds_filtered_df.fillna(0)

# Divide the columns by 100 and reassign
bond_funds_filtered_df['Yield_To_Maturity'] = bond_funds_filtered_df['Yield_To_Maturity'] / 100
bond_funds_filtered_df['Weighted_Avg_Coupon'] = bond_funds_filtered_df['Weighted_Avg_Coupon'] / 100

# Display the DataFrame with the updated columns
display(bond_funds_filtered_df.head())

,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,GOVZ,iShares 25+ Year Treasury STRIPS Bond ETF,0.65%,1.05%,-12.71%,0,0,-15.79%,"Apr 30, 2025","Sep 22, 2020",255201565,315911,https://www.blackrock.com/us/individual/produc...,0.0501,27.30,7.24,0.0000
1,IBGL,iShares® iBonds® Dec 2055 Term Treasury ETF,0,0,0,0,0,0,"Apr 30, 2025","Mar 25, 2025",3613848,342146,https://www.blackrock.com/us/individual/produc...,0.0491,29.74,3.48,0.0463
2,TLT,iShares 20+ Year Treasury Bond ETF,3.23%,5.46%,-6.03%,-9.43%,-0.87%,3.79%,"Apr 30, 2025","Jul 22, 2002",48796891937,239454,https://www.blackrock.com/us/individual/produc...,0.0497,25.48,3.38,0.0286
3,IBGB,iShares® iBonds® Dec 2045 Term Treasury ETF,0,0,0,0,0,0,"Apr 30, 2025","Mar 25, 2025",3634667,342124,https://www.blackrock.com/us/individual/produc...,0.0497,19.79,2.34,0.0318
4,ILTB,iShares Core 10+ Year USD Bond ETF,2.20%,6.73%,-1.03%,-3.92%,1.47%,3.87%,"Apr 30, 2025","Dec 08, 2009",591313158,239424,https://www.blackrock.com/us/individual/produc...,0.0566,21.76,2.34,0.0394


### Step 5: Import Quotes

In [8]:
bond_tickers = bond_funds_filtered_df['Ticker'].tolist()
print(bond_tickers)

['GOVZ', 'IBGL', 'TLT', 'IBGB', 'ILTB', 'IGLB', 'IBGA', 'TLH', 'ICVT', 'IGOV', 'LQD', 'ELQD', 'LQDI']


In [9]:
# Load and process data
bonds_daily_df = load_and_filter_data('data/datasets/daily_bond_quotes.csv', bond_tickers, start_date, end_date)
display(bonds_daily_df.head())

Found 10 of 13 tickers in data/datasets/daily_bond_quotes.csv
Missing tickers: ['IBGL', 'IBGB', 'IBGA']


,ELQD,GOVZ,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,,,
2020-05-18,NaN,NaN,53.14,53.90,49.03,59.37,109.30,21.06,143.08,142.32
2020-05-19,NaN,NaN,53.32,53.90,49.12,59.44,109.38,21.43,143.59,142.85
2020-05-20,NaN,NaN,54.00,54.77,49.37,60.18,110.59,21.54,143.83,143.28
2020-05-21,NaN,NaN,53.83,54.68,49.38,60.20,110.50,21.67,144.15,143.64
2020-05-22,NaN,NaN,54.24,54.88,49.15,60.49,110.50,21.61,144.60,144.50


### Step 6: Prepare Data (Drop Invalid Tickers)

#### Daily Quotes

In [10]:
# Identify columns with null values in first or last 50 rows
first_50_nulls = bonds_daily_df.head(50).isnull().any()
last_50_nulls = bonds_daily_df.tail(50).isnull().any()

# Columns to drop are those with nulls in first 50 OR last 50 rows
columns_to_drop = first_50_nulls | last_50_nulls
bad_columns = columns_to_drop[columns_to_drop].index.tolist()

print(f"Dropping {len(bad_columns)} columns with missing values in first/last 50 records: {bad_columns}")

# Drop those columns
bonds_daily_filtered_df = bonds_daily_df.loc[:, ~columns_to_drop]

# Display the cleaned dataframe
display(bonds_daily_filtered_df.head())

Dropping 2 columns with missing values in first/last 50 records: ['ELQD', 'GOVZ']


,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-05-18,53.14,53.90,49.03,59.37,109.30,21.06,143.08,142.32
2020-05-19,53.32,53.90,49.12,59.44,109.38,21.43,143.59,142.85
2020-05-20,54.00,54.77,49.37,60.18,110.59,21.54,143.83,143.28
2020-05-21,53.83,54.68,49.38,60.20,110.50,21.67,144.15,143.64
2020-05-22,54.24,54.88,49.15,60.49,110.50,21.61,144.60,144.50


#### Monthly Quotes

In [11]:
bonds_monthly_filtered_df = (bonds_daily_filtered_df.set_index(pd.to_datetime(bonds_daily_filtered_df.pop('Date')))
      if 'Date' in bonds_daily_filtered_df.columns else bonds_daily_filtered_df.copy())
bonds_monthly_filtered_df.index = pd.to_datetime(bonds_monthly_filtered_df.index)              
bonds_monthly_filtered_df = (bonds_monthly_filtered_df.resample('MS').last()
   .reset_index()
   .rename(columns={'index': 'Date'}))

bonds_monthly_filtered_df.set_index('Date', inplace=True)
display(bonds_monthly_filtered_df.head())

,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-05-01,54.97,55.43,49.74,60.61,111.62,21.93,143.98,142.79
2020-06-01,59.46,57.22,50.36,61.71,113.99,22.55,144.24,143.27
2020-07-01,63.86,60.35,52.75,65.17,117.53,23.49,148.27,149.62
2020-08-01,70.40,58.16,52.80,62.75,115.44,23.77,142.85,142.07
2020-09-01,67.84,58.00,52.31,62.61,114.93,23.62,143.94,143.17


### Step 7: Calculate Returns (Lognormal)

In [12]:
log_returns_df = np.log(bonds_daily_filtered_df / bonds_daily_filtered_df.shift(1))
log_returns_df = log_returns_df.dropna().sort_index(axis=0, ascending=True).sort_index(axis=1, ascending=True)
log_returns_df.head()

,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-05-19,0.003382,0.000000,0.001834,0.001178,0.000732,0.017416,0.003558,0.003717
2020-05-20,0.012673,0.016012,0.005077,0.012373,0.011002,0.005120,0.001670,0.003006
2020-05-21,-0.003153,-0.001645,0.000203,0.000332,-0.000814,0.006017,0.002222,0.002509
2020-05-22,0.007588,0.003651,-0.004669,0.004806,0.000000,-0.002773,0.003117,0.005969
2020-05-26,0.011001,-0.003102,0.003250,-0.005637,-0.001358,-0.003709,-0.007497,-0.013517


### Step 8: Calculate Bond Metrics

#### Bond Price (PV)

In [13]:
# Apply the function to your DataFrame
bond_funds_price_df = add_bond_prices_to_df(bond_funds_filtered_df)

# Display the updated DataFrame with bond prices
bond_funds_price_df[['Ticker', 'Weighted_Avg_Maturity', 'Weighted_Avg_Coupon', 
                      'Yield_To_Maturity', 'Bond_Price']]

,Ticker,Weighted_Avg_Maturity,Weighted_Avg_Coupon,Yield_To_Maturity,Bond_Price
0,GOVZ,27.30,0.000,0.050,25.901
1,IBGL,29.74,0.046,0.049,95.645
2,TLT,25.48,0.029,0.050,69.698
3,IBGB,19.79,0.032,0.050,77.616
4,ILTB,21.76,0.039,0.057,78.632
5,IGLB,22.10,0.046,0.060,83.077
6,IBGA,18.86,0.040,0.049,88.421
7,TLH,16.96,0.032,0.049,80.199
8,ICVT,2.93,0.021,0.029,97.769
9,IGOV,9.59,0.022,0.029,94.584


#### Modified Duration (D*)

In [14]:
bond_tickers = bonds_daily_filtered_df.columns.tolist()
bond_funds_filtered_df = bond_funds_filtered_df[bond_funds_filtered_df['Ticker'].isin(bond_tickers)].reset_index(drop=True)

# Apply the function to your DataFrame
bond_funds_duration_df = calculate_duration_for_bonds(bond_funds_filtered_df, end_date)

# Display the updated DataFrame
print(bond_tickers)
bond_funds_duration_df[['Ticker', 'Weighted_Avg_Maturity', 'Weighted_Avg_Coupon', 'Yield_To_Maturity', 'Bond_Price', 'Duration (D*)', 'Convexity']]

['ICVT', 'IGLB', 'IGOV', 'ILTB', 'LQD', 'LQDI', 'TLH', 'TLT']


,Ticker,Weighted_Avg_Maturity,Weighted_Avg_Coupon,Yield_To_Maturity,Bond_Price,Duration (D*),Convexity
0,TLT,25.48,0.029,0.050,69.698,17.861,3.38
1,ILTB,21.76,0.039,0.057,78.632,14.885,2.34
2,IGLB,22.10,0.046,0.060,83.077,14.584,2.17
3,TLH,16.96,0.032,0.049,80.199,12.869,1.88
4,ICVT,2.93,0.021,0.029,97.769,2.413,1.20
5,IGOV,9.59,0.022,0.029,94.584,8.531,1.11
6,LQD,12.82,0.044,0.054,90.748,9.740,1.11
7,LQDI,12.45,0.000,0.052,52.456,11.693,1.07


#### Price Sensitivity to Changes in YTM (-1%)

In [15]:
# Apply the function to your DataFrame
bond_funds_sensitivity_df = calculate_price_change_sensitivity(bond_funds_duration_df)
bond_funds_sensitivity_df[['Ticker', 'Bond_Price', 'Duration (D*)', 'Convexity', 'Price Sensitivity to YTM (-1%)']]

,Ticker,Bond_Price,Duration (D*),Convexity,Price Sensitivity to YTM (-1%)
0,TLT,69.698,17.861,3.38,0.1786
1,ILTB,78.632,14.885,2.34,0.1489
2,IGLB,83.077,14.584,2.17,0.1458
3,TLH,80.199,12.869,1.88,0.1287
4,ICVT,97.769,2.413,1.20,0.0241
5,IGOV,94.584,8.531,1.11,0.0853
6,LQD,90.748,9.740,1.11,0.0974
7,LQDI,52.456,11.693,1.07,0.1169


### Step 9: Select the Best Bond (Highest Price Sentivity to Changes in YTM)

In [16]:
# Find the row index where Price Sensitivity to YTM (-1%) is at its maximum
best_bond_ticker = bond_funds_sensitivity_df["Price Sensitivity to YTM (-1%)"].idxmax()

# Get the maximum value for display
max_sensitivity = bond_funds_sensitivity_df["Price Sensitivity to YTM (-1%)"].max()

print(f"Best Bond Ticker: {best_bond_ticker}")
print(f"Maximum Price Sensitivity to YTM (-1%): {max_sensitivity:.6f}")
bond_funds_sensitivity_df.loc[best_bond_ticker]

Best Bond Ticker: 0
Maximum Price Sensitivity to YTM (-1%): 0.178600


Ticker                                                                          TLT
Name                                             iShares 20+ Year Treasury Bond ETF
YTD (%)                                                                       3.23%
1Y (%)                                                                        5.46%
3Y (%)                                                                       -6.03%
5Y (%)                                                                       -9.43%
10Y (%)                                                                      -0.87%
Incept (%)                                                                    3.79%
Perf. as of                                                            Apr 30, 2025
Inception Date                                                         Jul 22, 2002
Net Assets                                                              48796891937
Product_ID                                                                  

### Step 10: Add Returns and Standard Deviation to DataFrame

In [17]:
# Step 1: Unpivot (melt) the log_returns_df
log_returns_long = log_returns_df.reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Log Return')

# Step 2: Ensure 'Log Return' column is numeric
log_returns_long['Log Return'] = pd.to_numeric(log_returns_long['Log Return'], errors='coerce')

# Step 3: Drop rows with NaN values in 'Log Return'
log_returns_long = log_returns_long.dropna(subset=['Log Return'])

# Step 4: Calculate statistics for each ticker
log_returns_stats = log_returns_long.groupby('Ticker').agg(
    Expected_Return=('Log Return', lambda x: ((1 + x).prod() ** (252 / len(x))) - 1),  # Geometric mean (annualized return)
    Standard_Deviation=('Log Return', lambda x: x.std() * np.sqrt(252))  # Annualized standard deviation
).reset_index()

# Step 6: Set Index
log_returns_stats.set_index('Ticker', inplace=True)

# Step 7: Merge with bond_funds_sensitivity_df
bond_funds_sensitivity_df = bond_funds_sensitivity_df.merge(
    log_returns_stats,
    on='Ticker',
    how='left'
)

# Step 7: Set Index
bond_funds_sensitivity_df.set_index('Ticker', inplace=True)

# Display the updated DataFrame
display(bond_funds_sensitivity_df)

,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,...,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon,Bond_Price,Duration (D*),Price Sensitivity to YTM (-1%),Expected_Return,Standard_Deviation
Ticker,,,,,,,,,,,,,,,,,,,,,
TLT,iShares 20+ Year Treasury Bond ETF,3.23%,5.46%,-6.03%,-9.43%,-0.87%,3.79%,"Apr 30, 2025","Jul 22, 2002",48796891937,...,https://www.blackrock.com/us/individual/produc...,0.050,25.48,3.38,0.029,69.698,17.861,0.1786,-0.107342,0.162255
ILTB,iShares Core 10+ Year USD Bond ETF,2.20%,6.73%,-1.03%,-3.92%,1.47%,3.87%,"Apr 30, 2025","Dec 08, 2009",591313158,...,https://www.blackrock.com/us/individual/produc...,0.057,21.76,2.34,0.039,78.632,14.885,0.1489,-0.048908,0.128049
IGLB,iShares 10+ Year Investment Grade Corporate Bo...,1.10%,6.19%,0.87%,-1.95%,2.09%,4.27%,"Apr 30, 2025","Dec 08, 2009",2498390313,...,https://www.blackrock.com/us/individual/produc...,0.060,22.10,2.17,0.046,83.077,14.584,0.1458,-0.028074,0.126794
TLH,iShares 10-20 Year Treasury Bond ETF,4.02%,7.73%,-2.63%,-6.71%,-0.35%,2.95%,"Apr 30, 2025","Jan 05, 2007",9898819870,...,https://www.blackrock.com/us/individual/produc...,0.049,16.96,1.88,0.032,80.199,12.869,0.1287,-0.078036,0.128536
ICVT,iShares Convertible Bond ETF,0.45%,13.09%,5.30%,10.75%,0,9.08%,"Apr 30, 2025","Jun 02, 2015",2472301830,...,https://www.blackrock.com/us/individual/produc...,0.029,2.93,1.20,0.021,97.769,2.413,0.0241,0.093883,0.150448
IGOV,iShares International Treasury Bond ETF,8.65%,9.56%,-0.71%,-3.32%,-0.86%,0.21%,"Apr 30, 2025","Jan 21, 2009",976165647,...,https://www.blackrock.com/us/individual/produc...,0.029,9.59,1.11,0.022,94.584,8.531,0.0853,-0.040130,0.096265
LQD,iShares iBoxx $ Investment Grade Corporate Bon...,2.22%,7.50%,2.61%,-0.17%,2.37%,4.41%,"Apr 30, 2025","Jul 22, 2002",29697566053,...,https://www.blackrock.com/us/individual/produc...,0.054,12.82,1.11,0.044,90.748,9.740,0.0974,-0.007896,0.087755
LQDI,iShares Inflation Hedged Corporate Bond ETF,2.32%,6.38%,2.12%,4.01%,0,3.95%,"Apr 30, 2025","May 08, 2018",90392195,...,https://www.blackrock.com/us/individual/produc...,0.052,12.45,1.07,0.000,52.456,11.693,0.1169,0.037848,0.088215


### Step 11: Export Data to Excel

In [18]:
# Check if best_bond_ticker is a numeric index instead of ticker name
if isinstance(best_bond_ticker, (int, np.integer)):
    # Get the actual ticker name from the dataframe
    if 'Ticker' in bond_funds_sensitivity_df.columns:
        best_bond_ticker = bond_funds_sensitivity_df.iloc[best_bond_ticker]['Ticker']
        print(f"Using ticker name: {best_bond_ticker}")
    else:
        # If Ticker is the index name
        best_bond_ticker = bond_funds_sensitivity_df.index[best_bond_ticker]
        print(f"Using ticker name from index: {best_bond_ticker}")

export_to_excel(output_file, {
    'bond': bond_funds_sensitivity_df,
    'daily_quotes': bonds_daily_filtered_df[[best_bond_ticker]],  
    'monthly_quotes': bonds_monthly_filtered_df[[best_bond_ticker]]   
})

Using ticker name from index: TLT
Updated sheet 'bond'
Successfully merged data into 'daily_quotes' sheet
Successfully merged data into 'monthly_quotes' sheet
Successfully exported all data to reports/portfolio-2025-05-16.xlsx
